To execute succesfully this notebook you should to install the following requirements:

* `langchain==0.1.11`
* `openai==1.13.3`
* `PyPDF2==3.0.1`
* `faiss-cpu==1.8.0`
* `tiktoken==0.6.0`
* `pandas`
* `pydantic==2.6.3`

In [3]:
from PyPDF2 import PdfReader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

In [5]:
# Load the openai key

from dotenv import load_dotenv
import os

load_dotenv()  # Carga las variables de entorno desde .env
open_api_key = os.getenv('OPENAI_API_KEY')

# Simple Call

In [7]:
# OpenAI parameters loaded into the class
llm = OpenAI(model_name='gpt-3.5-turbo-instruct', openai_api_key=open_api_key)

/tmp/ipykernel_8556/1805202783.py:2: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(model_name='gpt-3.5-turbo-instruct', openai_api_key=open_api_key)


In [9]:
our_query = "How many awards did David Palacio won?"
print(llm.invoke(our_query))



It is not specified who David Palacio is, so it is not possible to accurately answer this question. 


In [6]:
type(llm)

langchain_community.llms.openai.OpenAI

# LLM Call a PDF

In [14]:
data = PdfReader(r'../dpj_description.pdf')

In [15]:
print(len(data.pages))
print(data.metadata)

1
{'/Title': 'Documento sin título', '/Producer': 'Skia/PDF m132 Google Docs Renderer'}


In [17]:
combined_text = ''
for i, page in enumerate(data.pages):
    text = page.extract_text()
    if text:
        combined_text += text

print(type(combined_text))
print("\n")
print("La cantidad de caracteres que tiene el pdf corresponden a: " + str(len(combined_text)))

<class 'str'>


La cantidad de caracteres que tiene el pdf corresponden a: 652


In [19]:
text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 370,
    chunk_overlap = 20,
    length_function = len,
)

finalData = text_splitter.split_text(combined_text)
print(finalData)
print("\n")
print(type(finalData))
print(len(finalData))

['David\nPalacio\nis\na\nbilingual\nArtificial\nIntelligence\nEngineer\nwith\na\nstrong\nbackground\nin\nstatistics,\nmachine\nlearning,\nand\ndeep\nlearning.\nHe\nhas\nexpertise\nin\nsupervised\nlearning,\nneural\nnetworks,\nNLP,\nand\nlarge\nlanguage\nmodels\n(LLMs).\nDavid\nhas\ndeveloped\ninnovative\nAI\nsolutions\nacross\nvarious\nindustries,\nincluding\nfinancial\nestimations,\nhealthcare,\ngeological\nrisk', 'geological\nrisk\nprediction,\nand\ngenerative\nAI\nfor\ntreatment\nplans.\nHe\nhas\nextensive\nexperience\nin\nMLOps,\nhandling\nend-to-end\npipelines,\ndeployment,\nmonitoring,\nand\nCI/CD\nimplementations.\nPassionate\nabout\nleveraging\ncutting-edge\ntechnology,\nhe\nexcels\nat\ndriving\nsuccess\nand\nimproving\nteam\nperformance.']


<class 'list'>
2


In [20]:
#finalData[0]
finalData[1]

'geological\nrisk\nprediction,\nand\ngenerative\nAI\nfor\ntreatment\nplans.\nHe\nhas\nextensive\nexperience\nin\nMLOps,\nhandling\nend-to-end\npipelines,\ndeployment,\nmonitoring,\nand\nCI/CD\nimplementations.\nPassionate\nabout\nleveraging\ncutting-edge\ntechnology,\nhe\nexcels\nat\ndriving\nsuccess\nand\nimproving\nteam\nperformance.'

## FAISS

Facebook AI Similarity Search (Faiss) is a library for efficient similarity search and clustering of dense vectors. It contains algorithms that search in sets of vectors of any size, up to ones that possibly do not fit in RAM. It also contains supporting code for evaluation and parameter tuning. To see more information click [here](https://python.langchain.com/docs/integrations/vectorstores/faiss).

In [22]:
embeddings = OpenAIEmbeddings(openai_api_key=open_api_key)

In [23]:
documentSearch = FAISS.from_texts(finalData, embeddings)

In [24]:
chain = load_qa_chain(OpenAI(openai_api_key=open_api_key), chain_type="stuff")

/tmp/ipykernel_8556/3158837868.py:1: LangChainDeprecationWarning: This class is deprecated. See the following migration guides for replacements based on `chain_type`:
stuff: https://python.langchain.com/docs/versions/migrating_chains/stuff_docs_chain
map_reduce: https://python.langchain.com/docs/versions/migrating_chains/map_reduce_chain
refine: https://python.langchain.com/docs/versions/migrating_chains/refine_chain
map_rerank: https://python.langchain.com/docs/versions/migrating_chains/map_rerank_docs_chain

See also guides on retrieval and question-answering here: https://python.langchain.com/docs/how_to/#qa-with-rag
  chain = load_qa_chain(OpenAI(openai_api_key=open_api_key), chain_type="stuff")


In [25]:
our_query = "How old is David Palacio?"
docs = documentSearch.similarity_search(our_query)
chain.run(input_documents=docs, question=our_query)

/tmp/ipykernel_8556/3335512588.py:3: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain.run(input_documents=docs, question=our_query)


" I don't know."